# Multi Factor Optimization
1. Read factor ranking from files.
2. Assign weight for each factor
3. Rank again with assigned weights for each factors
4. Get Return based on rank


# Read factor ranking from files

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import os 

In [2]:

#Files1 =['gross profit_rank.csv','net profit after tax_rank.csv','revenue_rank.csv','operating profit_rank.csv','total equity_rank.csv','net cash flow from operating activities_rank.csv','net increase in cash and cash equivalents_rank.csv' ]
#Files2 =['1mmomentum_rank.csv','2mmomentum_rank.csv','3mmomentum_rank.csv','6mmomentum_rank.csv','11mmomentum_rank.csv',]
#rank1 = pd.read_csv('ranks/Valuation/net profit after tax_rank.csv') 
#rank2 = pd.read_csv('ranks/Momentum/2mmomentum_rank.csv')
monthly_ret = pd.read_csv('ranks/monthly_ret.csv')
opt_result=pd.DataFrame(columns=['weight1','weight2','cumul_return','mdd','return_mdd'])
file_result=pd.DataFrame(columns=['file1','file2','weight1','weight2','cumul_return','mdd','return_mdd'])
print(opt_result)


print(os.listdir('ranks/Quality'))
#Files2=os.listdir(os.listdir('ranks/Quality'))


Empty DataFrame
Columns: [weight1, weight2, cumul_return, mdd, return_mdd]
Index: []
['asset_turnover_rank.csv', 'cash_rank.csv', 'current assets_rank.csv', 'debt_asset_rank.csv', 'debt_equity_rank.csv', 'inventories_rank.csv', 'roa_rank.csv', 'roe_rank.csv']


In [3]:
def rank_opt(rank1, rank2, init, unit,opt_result):
    rank1 = rank1
    rank2 = rank2
    iterations = int(1/unit)
    
    counter=0
    
    for x in range(1, iterations) :
        weight1 = init + unit*counter
        weight2 = 1-weight1
        counter=counter + 1 
        rank_sum = rank1*weight1 + rank2*weight2
        rank_sum.yr_month=rank1.yr_month
        rank_rank=rank_sum.rank(axis=1, ascending=True, pct=True)
        rank_rank.yr_month=rank1.yr_month
        signal=rank_rank[rank_rank<=0.1].dropna(axis=1, how='all')
        signal[signal>0]=1
        signal.yr_month=rank1.yr_month
        #signal.to_csv('ranks/multi_factor_signal.csv')
        signal=signal.shift(1)
        mm_hold=signal.count(axis=1).max()
        profit_arr=(signal*monthly_ret[signal.columns.tolist()]).dropna(axis=1, how='all')


        profit_arr['daily_ret']=profit_arr.mean(axis=1)

        profit_arr['profit']=(profit_arr['daily_ret']+1).cumprod()
        daily_profit=profit_arr['daily_ret'].tolist()
        cumulative=profit_arr['profit'].dropna().tolist()
        #print(daily_profit)
        profit_arr['yr_month']=rank1.yr_month

        win_arr = [x for x in daily_profit if x > 0]
        loss_arr=[x for x in daily_profit if x < 0]
        win_rate = len(win_arr) / (len(loss_arr)+len(win_arr))
        dd_arr=[]
        for i in range(len(cumulative)):
            if i>0:
                dd_arr.append((max(cumulative[:i])- cumulative[i-1])/max(cumulative[:i]))
        max_dd=max(dd_arr)
        #print('weight1:{} and weight2:{}'.format(weight1,weight2))
        #print('CUMULATIVE PROFIT: {}'.format(round(cumulative[-1],4)))
        #print('MDD: {}' .format(max_dd))
        #print("mm_holding:{}------winrate:{}-----profit/MDD:{}".format(mm_hold, round(win_rate,4), round(cumulative[-1]/max_dd,4)))
        new_low ={'weight1':weight1, 'weight2':weight2,'cumul_return':round(cumulative[-1],4),'mdd':max_dd,'return_mdd':round(cumulative[-1]/max_dd,4)}
        opt_result = opt_result
        opt_result= opt_result.append(new_low,ignore_index=True)
    opt_result.to_csv("Ranks/opt_result.csv")
       
    return round(cumulative[-1],4)

In [4]:
#rank_opt(rank1, rank2, 0.01,0.01,opt_result)

In [17]:
def multi_opt(folder1, folder2,file_result):
    
    for file1 in os.listdir('ranks/'+folder1):
        for file2 in os.listdir('ranks/'+folder2):
            print(file1,file2)      
            #multi_opt(file1,file2,file_result)
            #print(opt_result)
            rank1 = pd.read_csv('ranks/'+folder1+'/'+file1) 
            rank2 = pd.read_csv('ranks/'+folder2+'/'+file2)
            monthly_ret = pd.read_csv('ranks/monthly_ret.csv')
            opt_result=pd.DataFrame(columns=['weight1','weight2','cumul_return','mdd','return_mdd'])
            rank_opt(rank1,rank2,0.01,0.01,opt_result)
            opt_result = pd.read_csv('ranks/opt_result.csv') 
            #print(opt_result.loc[opt_result['return_mdd'].idxmax()])
            file_new={'file1': file1, 'file2':file2}
            file_new =opt_result.loc[opt_result['return_mdd'].idxmax()]
            #print(file_new)
            file_result = file_result.append(file_new,ignore_index=True)
            global file_result2
            file_result.to_csv('file_result.csv')
            #print(file_result)
    return file_result

In [18]:
multi_opt('Valuation','Growth',file_result)

gross profit_rank.csv cash_qoq_rank.csv
gross profit_rank.csv cash_yoy_rank.csv
gross profit_rank.csv debt_qoq_rank.csv
gross profit_rank.csv debt_yoy_rank.csv
gross profit_rank.csv gross_profit_growth_rank.csv
gross profit_rank.csv inventory_qoq_rank.csv
gross profit_rank.csv inventory_yoy_rank.csv
gross profit_rank.csv npat_growth_rank.csv
gross profit_rank.csv operating_profit_growth_rank.csv
gross profit_rank.csv revenue_growth_rank.csv
net cash flow from operating activities_rank.csv cash_qoq_rank.csv
net cash flow from operating activities_rank.csv cash_yoy_rank.csv
net cash flow from operating activities_rank.csv debt_qoq_rank.csv
net cash flow from operating activities_rank.csv debt_yoy_rank.csv
net cash flow from operating activities_rank.csv gross_profit_growth_rank.csv
net cash flow from operating activities_rank.csv inventory_qoq_rank.csv
net cash flow from operating activities_rank.csv inventory_yoy_rank.csv
net cash flow from operating activities_rank.csv npat_growth_rank

,file1,file2,weight1,weight2,cumul_return,mdd,return_mdd,Unnamed: 0
0,NaN,NaN,0.88,0.12,11.2101,0.152401,73.5568,87.0
1,NaN,NaN,0.83,0.17,11.9265,0.170364,70.0061,82.0
2,NaN,NaN,0.92,0.08,10.9624,0.162689,67.3824,91.0
3,NaN,NaN,0.99,0.01,9.4754,0.152937,61.9567,98.0
4,NaN,NaN,0.40,0.60,15.6061,0.161093,96.8768,39.0
...,...,...,...,...,...,...,...,...
65,NaN,NaN,0.91,0.09,9.0464,0.164471,55.0030,90.0
66,NaN,NaN,0.94,0.06,8.8124,0.185862,47.4139,93.0
67,NaN,NaN,0.82,0.18,15.0143,0.155371,96.6347,81.0
68,NaN,NaN,0.80,0.20,15.7453,0.158810,99.1455,79.0


In [729]:
print(file_result)

Empty DataFrame
Columns: [file1, file2, weight1, weight2, cumul_return, mdd, return_mdd]
Index: []


In [672]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=opt_result['weight1'].tolist(), y=opt_result['cumul_return'].tolist(),
                                 mode='lines',
                                 name='cumulative'))

fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=opt_result['weight1'].tolist(), y=opt_result['return_mdd'].tolist(),
                                 mode='lines',
                                 name='cumulative'))

fig.show()

In [660]:
weight1 = 0.82
weight2 = 0.18

rank_sum = rank1*weight1 + rank2*weight2
rank_sum.yr_month=rank1.yr_month
rank_rank=rank_sum.rank(axis=1, ascending=True, pct=True)
rank_rank.yr_month=rank1.yr_month
signal=rank_rank[rank_rank<=0.1].dropna(axis=1, how='all')
signal[signal>0]=1
signal.yr_month=rank1.yr_month
#signal.to_csv('ranks/multi_factor_signal.csv')
signal=signal.shift(1)
mm_hold=signal.count(axis=1).max()
profit_arr=(signal*monthly_ret[signal.columns.tolist()]).dropna(axis=1, how='all')

signal.to_csv('equity_netprofitgrowth_signal.csv')
profit_arr['daily_ret']=profit_arr.mean(axis=1)

profit_arr['profit']=(profit_arr['daily_ret']+1).cumprod()
daily_profit=profit_arr['daily_ret'].tolist()
cumulative=profit_arr['profit'].dropna().tolist()
#print(daily_profit)
profit_arr['yr_month']=rank1.yr_month

win_arr = [x for x in daily_profit if x > 0]
loss_arr=[x for x in daily_profit if x < 0]
win_rate = len(win_arr) / (len(loss_arr)+len(win_arr))
dd_arr=[]
for i in range(len(cumulative)):
    if i>0:
        dd_arr.append((max(cumulative[:i])- cumulative[i-1])/max(cumulative[:i]))
max_dd=max(dd_arr)
print('weight1:{} and weight2:{}'.format(weight1,weight2))
print('CUMULATIVE PROFIT: {}'.format(round(cumulative[-1],4)))
print('MDD: {}' .format(max_dd))
print("mm_holding:{}------winrate:{}-----profit/MDD:{}".format(mm_hold, round(win_rate,4), round(cumulative[-1]/max_dd,4)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=profit_arr['yr_month'].astype(str).tolist(), y=profit_arr['profit'].tolist(),
                             mode='lines',
                             name='cumulative'))

fig.show()



weight1:0.82 and weight2:0.18
CUMULATIVE PROFIT: 10.0169
MDD: 0.16066480118441606
mm_holding:34------winrate:0.6795-----profit/MDD:62.3463


Calculation of profit based on generated signal
